In [1]:
import pandas as pd
import sqlite3

# 9.1 Reading data from SQL databases

So far we've only talked about reading data from CSV files. That's a pretty common way to store data, but there are many others! Pandas can read from HTML, JSON, SQL, Excel (!!!), HDF5, Stata, and a few other things. In this chapter we'll talk about reading data from SQL databases.

You can read data from a SQL database using the `pd.read_sql` function. `read_sql` will automatically convert SQL column names to DataFrame column names.

`read_sql` takes 2 arguments: a `SELECT` statement, and a database connection object. This is great because it means you can read from *any* kind of SQL database -- it doesn't matter if it's MySQL, SQLite, PostgreSQL, or something else.

This example reads from a SQLite database, but any other database would work the same way.

In [3]:
con = sqlite3.connect("../data/weather_2012.sqlite")
df = pd.read_sql("SELECT temp as T, date_time as Time, id as ID from weather_2012 LIMIT 3", con)
df

,T,Time,ID
0,-1.8,2012-01-01 00:00:00,1
1,-1.8,2012-01-01 01:00:00,2
2,-1.8,2012-01-01 02:00:00,3


`read_sql` doesn't automatically set the primary key (`id`) to be the index of the dataframe. You can make it do that by adding an `index_col` argument to `read_sql`. 

If you've used `read_csv` a lot, you may have seen that it has an `index_col` argument as well. This one behaves the same way.

In [4]:
df = pd.read_sql("SELECT * from weather_2012 LIMIT 3", con, index_col='id')
df

,date_time,temp
id,,
1,2012-01-01 00:00:00,-1.8
2,2012-01-01 01:00:00,-1.8
3,2012-01-01 02:00:00,-1.8


If you want your dataframe to be indexed by more than one column, you can give a list of columns to `index_col`:

In [5]:
df = pd.read_sql("SELECT * from weather_2012 LIMIT 3", con, 
                 index_col=['id', 'date_time'])
df

,,temp
id,date_time,
1,2012-01-01 00:00:00,-1.8
2,2012-01-01 01:00:00,-1.8
3,2012-01-01 02:00:00,-1.8


# 9.2 Writing to a SQLite database

Pandas has a `write_frame` function which creates a database table from a dataframe. Right now this only works for SQLite databases. Let's use it to move our 2012 weather data into SQL.

You'll notice that this function is in `pd.io.sql`. There are a ton of useful functions for reading and writing various kind of data in `pd.io`, and it's worth spending some time exploring them. ([see the documentation!](http://pandas.pydata.org/pandas-docs/stable/io.html))

In [6]:
weather_df = pd.read_csv('../data/weather_2012.csv')
con = sqlite3.connect("../data/test_db.sqlite")
con.execute("DROP TABLE IF EXISTS weather_2012")
weather_df.to_sql("weather_2012", con)

C:\Users\Hamahmi\Anaconda3\lib\site-packages\pandas\core\generic.py:2127: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


We can now read from the `weather_2012` table in  `test_db.sqlite`, and we see that we get the same data back:

In [7]:
con = sqlite3.connect("../data/test_db.sqlite")
df = pd.read_sql("SELECT * from weather_2012 LIMIT 3", con)
df

,index,Date/Time,Temp (C),Dew Point Temp (C),Rel Hum (%),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Weather
0,0,2012-01-01 00:00:00,-1.8,-3.9,86,4,8.0,101.24,Fog
1,1,2012-01-01 01:00:00,-1.8,-3.7,87,4,8.0,101.24,Fog
2,2,2012-01-01 02:00:00,-1.8,-3.4,89,7,4.0,101.26,"Freezing Drizzle,Fog"


The nice thing about having your data in a database is that you can do arbitrary SQL queries. This is cool especially if you're more familiar with SQL. Here's an example of sorting by the Weather column:

In [10]:
con = sqlite3.connect("../data/test_db.sqlite")
df = pd.read_sql("SELECT * from weather_2012 GROUP BY Weather", con)
df

,index,Date/Time,Temp (C),Dew Point Temp (C),Rel Hum (%),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Weather
0,8756,2012-12-30 20:00:00,-13.8,-16.5,80,24,25.0,101.52,Clear
1,8766,2012-12-31 06:00:00,-9.7,-11.7,85,4,25.0,101.23,Cloudy
2,8545,2012-12-22 01:00:00,1.2,0.1,92,30,12.9,97.84,Drizzle
3,8482,2012-12-19 10:00:00,0.6,0.2,97,15,2.4,100.42,"Drizzle,Fog"
4,8433,2012-12-17 09:00:00,0.4,-0.7,92,20,4.0,100.79,"Drizzle,Ice Pellets,Fog"
5,8490,2012-12-19 18:00:00,1.2,0.1,92,9,9.7,101.15,"Drizzle,Snow"
6,8547,2012-12-22 03:00:00,1.1,0.1,93,28,9.7,97.93,"Drizzle,Snow,Fog"
7,8722,2012-12-29 10:00:00,-12.0,-13.3,90,7,6.4,101.15,Fog
8,8424,2012-12-17 00:00:00,-2.3,-3.3,93,24,12.9,101.02,Freezing Drizzle
9,8261,2012-12-10 05:00:00,-1.9,-2.8,94,33,4.0,100.55,"Freezing Drizzle,Fog"


If you have a PostgreSQL database or MySQL database, reading from it works exactly the same way as reading from a SQLite database. You create a connection using `psycopg2.connect()` or `MySQLdb.connect()`, and then use

`pd.read_sql("SELECT whatever from your_table", con)`

# 9.3 Connecting to other kinds of database

To connect to a MySQL database:

*Note: For these to work, you will need a working MySQL / PostgreSQL database, with the correct localhost, database name, etc.*

To connect to a PostgreSQL database:

<style>
    @font-face {
        font-family: "Computer Modern";
        src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');
    }
    div.cell{
        width:800px;
        margin-left:16% !important;
        margin-right:auto;
    }
    h1 {
        font-family: Helvetica, serif;
    }
    h4{
        margin-top:12px;
        margin-bottom: 3px;
       }
    div.text_cell_render{
        font-family: Computer Modern, "Helvetica Neue", Arial, Helvetica, Geneva, sans-serif;
        line-height: 145%;
        font-size: 130%;
        width:800px;
        margin-left:auto;
        margin-right:auto;
    }
    .CodeMirror{
            font-family: "Source Code Pro", source-code-pro,Consolas, monospace;
    }
    .text_cell_render h5 {
        font-weight: 300;
        font-size: 22pt;
        color: #4057A1;
        font-style: italic;
        margin-bottom: .5em;
        margin-top: 0.5em;
        display: block;
    }
    
    .warning{
        color: rgb( 240, 20, 20 )
        }  